Aleksandra Bednarczuk

# Predictors of elderly citizens willingness to stay in the neighbourhood
# Logistic regression

Original dataset, including barris among predictors

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
from imblearn.over_sampling import SMOTE

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import r2_score

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder

from scipy import stats

In [3]:
data = pd.read_pickle("survey_transformed2.pkl")

FileNotFoundError: [Errno 2] No such file or directory: 'survey_transformed2.pkl'

In [ ]:
data.reset_index(drop=True, inplace=True)

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.columns.tolist()

In [ ]:
for i in data.columns.tolist():
    display(pd.crosstab(data[i], data.Residential_satisfaction))

In [35]:
data.columns.tolist()

['Residential_satisfaction',
 'District',
 'Woman',
 'BCN_evolution',
 'BCN_future',
 'BCN_security',
 'BCN_management',
 'BCN_transport',
 'GARBAGE COLLECTION',
 'STREETS CLEANING',
 'NOISE',
 'GREEN AREAS',
 'PUBLIC LIGHTING',
 'ASPHALT AND STREET CONSERVATION',
 'CIRCULATION AND TRAFFIC',
 'BUS',
 'CITIZEN SECURITY IN THE CITY AS A WHOLE']

In [36]:
x_var = [#'District',
 'Woman',
 'BCN_evolution',
 'BCN_future',
 'BCN_management',
 'BCN_transport',
 'GARBAGE COLLECTION',
 'STREETS CLEANING',
 'NOISE',
 'GREEN AREAS',
 'PUBLIC LIGHTING',
 'ASPHALT AND STREET CONSERVATION',
 'CIRCULATION AND TRAFFIC',
 'BUS',
 'CITIZEN SECURITY IN THE CITY AS A WHOLE']

In [37]:
y = data['Residential_satisfaction']

In [38]:
x = pd.get_dummies(data[x_var])

In [39]:
x.columns.tolist()

['Woman',
 'GARBAGE COLLECTION',
 'STREETS CLEANING',
 'NOISE',
 'GREEN AREAS',
 'PUBLIC LIGHTING',
 'ASPHALT AND STREET CONSERVATION',
 'CIRCULATION AND TRAFFIC',
 'BUS',
 'CITIZEN SECURITY IN THE CITY AS A WHOLE',
 'BCN_evolution_IT HAS IMPROVED',
 'BCN_evolution_IT HAS WORSENED',
 'BCN_evolution_IT IS THE SAME',
 'BCN_future_IT WILL IMPROVE',
 'BCN_future_IT WILL STAY THE SAME',
 'BCN_future_IT WILL WORSEN',
 'BCN_management_BAD',
 'BCN_management_GOOD',
 'BCN_management_NEITHER BAD, NOR GOOD',
 'BCN_transport_BAD',
 'BCN_transport_GOOD',
 'BCN_transport_NEITHER BAD, NOR GOOD']

In [40]:
for i in x.columns.tolist():
    if i[-4:] == '_BAD':
        del x[i]

#del x['District_EIXAMPLE']

In [ ]:
import seaborn as sns # just a conventional alias, don't know why
sns.set(style="white")

corr = x.corr('spearman')

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(16, 16))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, #annot = corr.round(2), 
            linewidths=.5, cbar_kws={"shrink": .5})

In [41]:
x = sm.add_constant(x)

# Logit

In [42]:
glm_logit = sm.GLM(y, x, family=sm.families.Binomial())
glm_logit_results = glm_logit.fit()
glm_logit_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                    Generalized Linear Model Regression Results                     
====================================================================================
Dep. Variable:     Residential_satisfaction   No. Observations:                 1169
Model:                                  GLM   Df Residuals:                     1150
Model Family:                      Binomial   Df Model:                           18
Link Function:                        logit   Scale:                          1.0000
Method:                                IRLS   Log-Likelihood:                -377.46
Date:                      Sun, 09 May 2021   Deviance:                       754.92
Time:                              14:19:37   Pearson chi2:                 1.20e+03
No. Iterations:                          12                                         
Covariance Type:                  nonrobust                                         
===========================================================================================================
                                              coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------
const                                       0.2699      0.222      1.215      0.225      -0.166       0.705
Woman                                       0.5145      0.197      2.607      0.009       0.128       0.901
GARBAGE COLLECTION                          0.4762      0.228      2.089      0.037       0.029       0.923
STREETS CLEANING                            0.1194      0.233      0.512      0.609      -0.338       0.577
NOISE                                       0.0191      0.220      0.087      0.931      -0.412       0.450
GREEN AREAS                                 0.1865      0.214      0.871      0.384      -0.233       0.606
PUBLIC LIGHTING                            -0.0414      0.211     -0.196      0.845      -0.455       0.372
ASPHALT AND STREET CONSERVATION             0.5569      0.246      2.261      0.024       0.074       1.040
CIRCULATION AND TRAFFIC                    -0.1527      0.242     -0.632      0.527      -0.626       0.321
BUS                                        -0.0517      0.218     -0.237      0.812      -0.478       0.375
CITIZEN SECURITY IN THE CITY AS A WHOLE    -0.0745      0.263     -0.283      0.777      -0.590       0.441
BCN_evolution_IT HAS IMPROVED               0.4023      0.246      1.637      0.102      -0.079       0.884
BCN_evolution_IT HAS WORSENED              -0.3780      0.144     -2.619      0.009      -0.661      -0.095
BCN_evolution_IT IS THE SAME                0.2457      0.207      1.187      0.235      -0.160       0.652
BCN_future_IT WILL IMPROVE                  0.6823      0.157      4.342      0.000       0.374       0.990
BCN_future_IT WILL STAY THE SAME            0.0832      0.166      0.503      0.615      -0.241       0.408
BCN_future_IT WILL WORSEN                  -0.4957      0.150     -3.295      0.001      -0.791      -0.201
BCN_management_GOOD                         1.1661      0.329      3.542      0.000       0.521       1.811
BCN_management_NEITHER BAD, NOR GOOD        0.6699      0.248      2.706      0.007       0.185       1.155
BCN_transport_GOOD                          0.1073      0.295      0.364      0.716      -0.470       0.685
BCN_transport_NEITHER BAD, NOR GOOD         0.0077      0.394      0.019      0.985      -0.765       0.780
===========================================================================================================
"""

In [43]:
glm_logit_results.aic

792.9152349333629

In [44]:
glm_logit_results.bic

-7368.574320759516

In [45]:
glm_logit_results.pvalues.sort_values(ascending=False)

BCN_transport_NEITHER BAD, NOR GOOD        0.984501
NOISE                                      0.930792
PUBLIC LIGHTING                            0.844549
BUS                                        0.812413
CITIZEN SECURITY IN THE CITY AS A WHOLE    0.776991
BCN_transport_GOOD                         0.715768
BCN_future_IT WILL STAY THE SAME           0.615113
STREETS CLEANING                           0.608583
CIRCULATION AND TRAFFIC                    0.527196
GREEN AREAS                                0.383808
BCN_evolution_IT IS THE SAME               0.235397
const                                      0.224517
BCN_evolution_IT HAS IMPROVED              0.101629
GARBAGE COLLECTION                         0.036718
ASPHALT AND STREET CONSERVATION            0.023732
Woman                                      0.009141
BCN_evolution_IT HAS WORSENED              0.008818
BCN_management_NEITHER BAD, NOR GOOD       0.006815
BCN_future_IT WILL WORSEN                  0.000985
BCN_manageme

In [ ]:
for i in ['Living_with_disabled_person', 'Financial_situation_GOOD','Disability']:
    del x[i]

In [ ]:
glm_logit = sm.GLM(y, x, family=sm.families.Binomial())
glm_logit_results = glm_logit.fit()
glm_logit_results.summary()

In [ ]:
glm_logit_results.aic

In [ ]:
glm_logit_results.bic

In [ ]:
glm_logit_results.pvalues.sort_values(ascending=False)

In [ ]:
for i in ['Car', 'CIRCULATION AND TRAFFIC', 'NOISE', 'PUBLIC LIGHTING']:
    del x[i]

In [ ]:
glm_logit = sm.GLM(y, x, family=sm.families.Binomial())
glm_logit_results = glm_logit.fit()
glm_logit_results.summary()

In [ ]:
glm_logit_results.aic

In [ ]:
glm_logit_results.bic

In [ ]:
glm_logit_results.pvalues.sort_values(ascending=False)

In [ ]:
for i in ['STREETS CLEANING']:
    del x[i]

In [ ]:
glm_logit = sm.GLM(y, x, family=sm.families.Binomial())
glm_logit_results = glm_logit.fit()
glm_logit_results.summary()

In [ ]:
glm_logit_results.aic

In [ ]:
glm_logit_results.bic

In [ ]:
glm_logit_results.pvalues.sort_values(ascending=False)

In [ ]:
def glm_classification_table(results):
    df_classification_table = pd.DataFrame(y).join(pd.DataFrame(results.predict(x).round()))
    df_classification_table.rename(columns={'BCN_residential_satisfaction': 'y', 0: 'y_predict'}, inplace=True)
    return pd.crosstab(df_classification_table['y'], df_classification_table['y_predict'])

In [ ]:
glm_classification_table(glm_logit_results)

In [ ]:
def plot_roc_auc(results):
    y_predict = results.predict(x)
    roc_auc = roc_auc_score(y, y_predict)
    fpr, tpr, thresholds = roc_curve(y, y_predict)
    plt.figure()
    plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
plot_roc_auc(glm_logit_results)

In [ ]:
def odds_ratios(results):
    return np.exp(results.params)

In [ ]:
odds_ratios(glm_logit_results)

# Probit

In [ ]:
glm_probit = sm.GLM(y, x, family=sm.families.Binomial(link=sm.genmod.families.links.probit))
glm_probit_results = glm_probit.fit()
glm_probit_results.summary()

In [ ]:
glm_probit_results.aic

In [ ]:
glm_probit_results.bic

In [ ]:
glm_classification_table(glm_probit_results)

In [ ]:
plot_roc_auc(glm_probit_results)

In [ ]:
odds_ratios(glm_probit_results)

# Scobit

In [ ]:
glm_scobit = sm.GLM(y, x, family=sm.families.Binomial(link=sm.genmod.families.links.cloglog))
glm_scobit_results = glm_scobit.fit()
glm_scobit_results.summary()

In [ ]:
glm_scobit_results.aic

In [ ]:
glm_scobit_results.bic

In [ ]:
glm_classification_table(glm_scobit_results)

In [ ]:
plot_roc_auc(glm_scobit_results)

In [ ]:
odds_ratios(glm_scobit_results)